#### Imports

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from utils import *

np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)

2023-08-12 18:51:25.354619: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

#### GPU settings

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0) 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #comment this while debugging

#### Dataset curator

In [4]:
def tts_bangladesh():
    train_file_path = 'data/ban_train.hdf5' # Replace wih your own path
    test_file_path = 'data/ban_val.hdf5'    # Replace wih your own path
    
    X_train, Y_train = load_hdf5_data(train_file_path)
    X_test, Y_test = load_hdf5_data(test_file_path)
    
    X_train, X_pool, Y_train, Y_pool = train_test_split(X_train, Y_train, test_size=0.9, stratify=Y_train, random_state=0)
  
    X_train = np.array(imgs_input_fn(X_train))
    X_pool = np.array(imgs_input_fn(X_pool))
    X_test = np.array(imgs_input_fn(X_test))
    
    return X_train, X_pool, X_test, Y_train, Y_pool, Y_test

In [5]:
def tts_india():
    file_path = 'data/delhi_NCR.h5' # replace with the path to the dataset
    
    X_train, Y_train = load_hdf5_data(file_path)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train, random_state=0)
    X_train, X_pool, Y_train, Y_pool = train_test_split(X_train, Y_train, test_size=0.99, stratify = Y_train, random_state=0)
  
    X_train = np.array(imgs_input_fn(X_train))
    X_pool = np.array(imgs_input_fn(X_pool))
    X_test = np.array(imgs_input_fn(X_test))

    return X_train, X_pool, X_test, Y_train, Y_pool, Y_test

In [6]:
def tts_transfer():
    india_file_path = 'data/delhi_NCR.h5' #replace with the path to dataset
    bangladesh_file_path = 'data/ban_train.hdf5' #replace with the path to dataset
    
    X_train_bangladesh, Y_train_bangladesh = load_hdf5_data(bangladesh_file_path)
    X_train_india, Y_train_india = load_hdf5_data(india_file_path)

    X_train, X_test, Y_train, Y_test = train_test_split(X_train_india, Y_train_india, test_size=0.2, stratify=Y_train_india, random_state=0)
    X_train, X_pool, Y_train, Y_pool = train_test_split(X_train, Y_train, test_size=0.99, stratify = Y_train, random_state=0)
  
    X_train = np.array(imgs_input_fn(np.concatenate((X_train, X_train_bangladesh), axis = 0)))
    X_pool = np.array(imgs_input_fn(X_pool))
    X_test = np.array(imgs_input_fn(X_test))
    
    return X_train, X_pool, X_test, Y_train, Y_pool, Y_test

#### Acquisition Functions

In [7]:
def bald_score(predictions):
    expected_entropy = -np.sum(np.mean(predictions, axis = 0)*np.log2(np.mean(predictions, axis = 0)), axis = 1)
    predictive_entropy = np.mean(-np.sum(predictions*np.log2(predictions), axis = 2), axis = 0)
    return abs(expected_entropy-predictive_entropy)

In [8]:
def entropy_score(predictions):
    entropies = np.mean(-np.sum(predictions * np.log(predictions), axis=2), axis = 0)
    return entropies

#### Main loop

In [9]:
# Active learning loop parameters
num_iterations = 100
batch_size = 32
num_label = 1
num_mc_samples = 10

In [11]:
def al_loop(entropy, case, num_iterations = 100, batch_size = 32, num_label = 1, num_mc_samples = 10, rand_num = 0):
    # Reinitialise model
    model = resnet_model(learning_rate=0.00002)

    # Creating the required dataset
    if(case == "bangladesh"):
        X_train, X_pool, X_test, Y_train, Y_pool, Y_test = tts_bangladesh()
        initial_epoch = 21        
        subsequent_epoch = 5        
    elif(case == "india"):
        X_train, X_pool, X_test, Y_train, Y_pool, Y_test = tts_india()
        initial_epoch = 100
        subsequent_epoch = 5
    elif(case == 'transfer'):
        X_train, X_pool, X_test, Y_train, Y_pool, Y_test = tts_transfer()
        initial_epoch = 18
        subsequent_epoch = 4

    # Lists to store the test loss and accuracy for each iteration
    test_loss_list = []
    test_accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    a = X_pool.shape[0]
    shape = (10, 100)
    random_array = np.random.randint(0, a , shape)

    # save_folder = "acquired_"
    # save_folder += case
    # save_folder += entropy
    # if not os.path.exists(save_folder):
    #     os.makedirs(save_folder)

    for iteration in tqdm(range(num_iterations)):
        print(f"Iteration {iteration + 1}")
        
        # Train the model on the labeled data
        if(iteration==0):
            model.fit(X_train, Y_train, batch_size=batch_size, epochs=initial_epoch, verbose=0)
        else:
            model.fit(X_train, Y_train, batch_size=batch_size, epochs=subsequent_epoch, verbose=0)

        # Evaluate the model on the test set and store the metrics
        test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
        test_loss_list.append(test_loss)
        test_accuracy_list.append(test_accuracy)
        
        # # Get predictions on the test set
        Y_pred = np.round(model.predict(X_test))

        # Calculate precision and recall for the test set
        precision = precision_score(Y_test, Y_pred)
        recall = recall_score(Y_test, Y_pred)
        f1 = f1_score(Y_test,Y_pred)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)

        # Forward passes
        
        predictions = [model(X_pool, training = True) for _ in range(num_mc_samples)]
        predictions = np.array(predictions)
        complementary_predictions = 1 - predictions
        predictions = np.concatenate((predictions, complementary_predictions), axis=2)
        selected_indices = [random_array[rand_num][iteration]]

        if(entropy == 'bald'):
            scores = bald_score(predictions)
            pool_indices = np.arange(len(X_pool))
            selected_indices = pool_indices[np.argsort(scores)][-num_label:]
        elif entropy == 'entropy':
            scores = entropy_score(predictions)
            pool_indices = np.arange(len(X_pool))
            selected_indices = pool_indices[np.argsort(scores)][-num_label:]

        # Move the selected samples from the pool to the labeled set
        X_train = np.concatenate((X_train, X_pool[selected_indices]), axis=0)
        Y_train = np.concatenate((Y_train, Y_pool[selected_indices]), axis=0)

        # Save the selected images
        
        # image_data = X_pool[selected_indices[0]]  # Assuming X_pool contains image data
        # image = Image.fromarray((image_data))
        # selected_image_path = os.path.join(save_folder, f"iteration{iteration}.jpg")
        # image.save(selected_image_path)
        
        # Remove the selected samples from the pool
        if(entropy!='random'):
            X_pool = np.delete(X_pool, selected_indices, axis=0)
            Y_pool = np.delete(Y_pool, selected_indices, axis=0)
        
        print("Precision: ",precision)
        print("Recall: ",recall)
        print("Accuracy: ",test_accuracy)
    model.save('ALBald_model.h5')


    # Print the metrics at each iteration
    for i, (loss, accuracy, precision, recall) in enumerate(zip(test_loss_list, test_accuracy_list, precision_list, recall_list), 1):
        print(f"Iteration {i}: Test Loss = {loss:.4f}, Test Accuracy = {accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}")

    d1 = pd.DataFrame(precision_list)
    d2 = pd.DataFrame(recall_list)
    d3 = pd.DataFrame(test_accuracy_list)
    d4 = pd.DataFrame(f1_score_list)

    # Create a directory named "bald"
    os.makedirs(f"{case}_{entropy}", exist_ok=True)

    # Save DataFrames to CSV files in the "bald" directory
    d1.to_csv(f"{case}_{entropy}/precision_{rand_num}.csv", index=False)
    d2.to_csv(f"{case}_{entropy}/recall_{rand_num}.csv", index=False)
    d3.to_csv(f"{case}_{entropy}/test_accuracy_{rand_num}.csv", index=False)
    d4.to_csv(f"{case}_{entropy}/f1_score_{rand_num}.csv", index=False)

    return test_loss_list, test_accuracy_list, precision_list, recall_list, f1_score_list

In [12]:
test_loss_list, test_accuracy_list, precision_list, recall_list, f1_score_list = al_loop('bald','india')

Images shape:  (2762, 256, 256, 3) Images dtype:  uint8
Labels shape:  (2762,) Labels dtype:  float64


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 1


2023-08-12 18:51:39.811827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8903
2023-08-12 18:51:40.020161: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-12 18:51:40.468011: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f2c8941bf90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-12 18:51:40.468076: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-08-12 18:51:40.473720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-12 18:51:40.646922: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

18/18 [==============================] - 1s 14ms/step


...

Precision:  0.9363057324840764
Recall:  0.9607843137254902
Accuracy:  0.9710668921470642
Iteration 100
18/18 [==============================] - 0s 14ms/step


100%|██████████| 100/100 [43:02<00:00, 25.82s/it]

Precision:  0.9459459459459459
Recall:  0.9150326797385621
Accuracy:  0.9620253443717957


Iteration 1: Test Loss = 0.3448, Test Accuracy = 0.8336, Precision = 0.7850, Recall = 0.5490
Iteration 2: Test Loss = 0.3317, Test Accuracy = 0.8517, Precision = 0.8034, Recall = 0.6144
Iteration 3: Test Loss = 0.3130, Test Accuracy = 0.8590, Precision = 0.7698, Recall = 0.6993
Iteration 4: Test Loss = 0.3104, Test Accuracy = 0.8644, Precision = 0.7349, Recall = 0.7974
Iteration 5: Test Loss = 0.3239, Test Accuracy = 0.8644, Precision = 0.7167, Recall = 0.8431
Iteration 6: Test Loss = 0.3204, Test Accuracy = 0.8644, Precision = 0.7167, Recall = 0.8431
Iteration 7: Test Loss = 0.2933, Test Accuracy = 0.8752, Precision = 0.7500, Recall = 0.8235
Iteration 8: Test Loss = 0.2870, Test Accuracy = 0.8662, Precision = 0.7970, Recall = 0.6928
Iteration 9: Test Loss = 0.2872, Test Accuracy = 0.8626, Precision = 0.8130, Recall = 0.6536
Iteration 10: Test Loss = 0.2769, Test Accuracy = 0.8644, Precision = 0.8095, Recall = 0.6667
Iteration 11: Test Loss = 0.2729, Test Accuracy = 0.8644, Precision =

: 